In [1]:
import requests
import json
import pandas as pd

data = []

def get_full_json(url):

    a = requests.get(url)

    if a.status_code != 200:
        print(a.status_code)

        with open('data_req.json', 'a') as outfile:
            json.dump(data, outfile)

        return "httperror"

    cats = a.json()
    
    #for p in cats["objects"]["results"]:
    for p in cats["objects"]:

        data.append(p)

    if cats["meta"]["next"] == None:
        with open('data_req.json', 'a') as outfile:
            json.dump(data, outfile)
        return "httperror"


def generate_next_url(url):
    number = 0

    while True:
        full_url = url + str(number)

        number += 50
        
        #print("adding", full_url)
        yield full_url

for url in generate_next_url("https://fragdenstaat.de/api/v1/category/?limit=50&offset="):
    b = get_full_json(url)
    if b == "httperror":
        #print("http error. Stopping")
        break

In [2]:
cats = pd.DataFrame.from_dict(data)
cats.head()

,children,depth,id,is_topic,name,parent,slug
0,[],1,181,True,Arbeit,None,arbeit
1,[https://fragdenstaat.de/api/v1/category/210/],1,12,True,Auswärtiges,None,auswartiges
2,[],2,210,False,Europa,https://fragdenstaat.de/api/v1/category/12/,europa
3,[],1,18,True,Bau,None,bau
4,"[https://fragdenstaat.de/api/v1/category/196/,...",1,185,True,Bildung,None,bildung


In [3]:
cats = cats.drop(["children", "depth", "is_topic", "name", "parent"], axis=1)
cats.iloc[28:]

,id,slug
28,1,justiz
29,219,kommunales
30,183,kultur
31,223,kunst
32,2,landwirtschaft
33,227,legislative
34,184,medien
35,209,migration
36,236,auslaenderbehoerde
37,213,fluchtlinge


In [4]:
publicbody = pd.read_csv("publicbody.csv", index_col="id")
publicbody.loc[publicbody['categories'] == 'entwicklung']

,slug,categories,classification,name,number_of_requests,other_names,parent,regions,request_note,request_note_html,resource_uri,root,site_url,bundesland,bundesland_id,region
id,,,,,,,,,,,,,,,,
95,bundesministerium-fur-wirtschaftliche-zusammen...,entwicklung,ministerium,Bundesministerium für wirtschaftliche Zusammen...,203,"BMZ,Entwicklungsminister,Gerd Müller,Bundeswir...",NaN,[],NaN,NaN,https://fragdenstaat.de/api/v1/publicbody/95/,NaN,https://fragdenstaat.de/behoerde/bundesministe...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
307,centrum-fur-internationale-migration-und-entwi...,entwicklung,integration,Centrum für internationale Migration und Entwi...,0,Centrum für internationale Migration und Entwi...,https://fragdenstaat.de/api/v1/publicbody/88/,[],Bundesministerium für wirtschaftliche Zusammen...,<p>Bundesministerium für wirtschaftliche Zusam...,https://fragdenstaat.de/api/v1/publicbody/307/,https://fragdenstaat.de/api/v1/publicbody/95/,https://fragdenstaat.de/behoerde/centrum-fur-i...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
340,deutsche-gesellschaft-fur-internationale-zusam...,entwicklung,giz,Deutsche Gesellschaft für Internationale Zusam...,59,NaN,NaN,[],Laut einem [Gutachten des Bundestags](https://...,"<p>Laut einem <a href=""https://sehrgutachten.d...",https://fragdenstaat.de/api/v1/publicbody/340/,NaN,https://fragdenstaat.de/behoerde/deutsche-gese...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
342,deutsche-investitions-und-entwicklungsgesellsc...,entwicklung,wirtschaftsforderung,Deutsche Investitions- und Entwicklungsgesells...,0,NaN,NaN,[],NaN,NaN,https://fragdenstaat.de/api/v1/publicbody/342/,NaN,https://fragdenstaat.de/behoerde/deutsche-inve...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
395,deutsches-rotes-kreuz,entwicklung,katastrophenschutz,Deutsches Rotes Kreuz,7,NaN,NaN,[],NaN,NaN,https://fragdenstaat.de/api/v1/publicbody/395/,NaN,https://fragdenstaat.de/behoerde/deutsches-rot...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
15850,engagement-global-ggmbh,entwicklung,wirtschaftsforderung,Engagement Global gGmbH,1,Engagement Global gGmbH\r\r\nService für Entwi...,NaN,[],NaN,NaN,https://fragdenstaat.de/api/v1/publicbody/15850/,NaN,https://fragdenstaat.de/behoerde/engagement-gl...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/
626,kfw-entwicklungsbank,entwicklung,entwicklungsbank,KfW-Entwicklungsbank,3,NaN,https://fragdenstaat.de/api/v1/publicbody/656/,[],NaN,NaN,https://fragdenstaat.de/api/v1/publicbody/626/,https://fragdenstaat.de/api/v1/publicbody/84/,https://fragdenstaat.de/behoerde/kfw-entwicklu...,bund,1,https://fragdenstaat.de/api/v1/georegion/1/


dict basteln, das jedem slug eine kategorie zuordnet. das dict dann in anderem dataframe verwenden...

In [5]:
cats.head()

,id,slug
0,181,arbeit
1,12,auswartiges
2,210,europa
3,18,bau
4,185,bildung


In [6]:
cats.loc[cats.id == 181, "stat_classification"] = "arbeit-soziales"
cats.loc[cats.id == 182, "stat_classification"] = "arbeit-soziales"

cats.loc[cats.id == 200, "stat_classification"] = "integration-migration-flucht"
cats.loc[cats.id == 209, "stat_classification"] = "integration-migration-flucht"
cats.loc[cats.id == 236, "stat_classification"] = "integration-migration-flucht"
cats.loc[cats.id == 213, "stat_classification"] = "integration-migration-flucht"

cats.loc[cats.id == 12, "stat_classification"] = "auswaertiges"

cats.loc[cats.id == 185, "stat_classification"] = "bildung-forschung"
cats.loc[cats.id == 196, "stat_classification"] = "bildung-forschung"
cats.loc[cats.id == 197, "stat_classification"] = "bildung-forschung"
cats.loc[cats.id == 212, "stat_classification"] = "bildung-forschung"
cats.loc[cats.id == 186, "stat_classification"] = "bildung-forschung"
cats.loc[cats.id == 199, "stat_classification"] = "bildung-forschung"

cats.loc[cats.id == 2, "stat_classification"] = "ernahrung-landwirtschaft-verbraucherschutz"
cats.loc[cats.id == 13, "stat_classification"] = "ernahrung-landwirtschaft-verbraucherschutz"
cats.loc[cats.id == 189, "stat_classification"] = "ernahrung-landwirtschaft-verbraucherschutz"
cats.loc[cats.id == 231, "stat_classification"] = "ernahrung-landwirtschaft-verbraucherschutz"
cats.loc[cats.id == 220, "stat_classification"] = "ernahrung-landwirtschaft-verbraucherschutz"

cats.loc[cats.id == 211, "stat_classification"] = "familie-senioren-frauen-jugend"
cats.loc[cats.id == 193, "stat_classification"] = "familie-senioren-frauen-jugend"
cats.loc[cats.id == 203, "stat_classification"] = "familie-senioren-frauen-jugend"
cats.loc[cats.id == 192, "stat_classification"] = "familie-senioren-frauen-jugend"
cats.loc[cats.id == 201, "stat_classification"] = "familie-senioren-frauen-jugend"
cats.loc[cats.id == 194, "stat_classification"] = "familie-senioren-frauen-jugend"

cats.loc[cats.id == 8, "stat_classification"] = "gesundheit"
cats.loc[cats.id == 89, "stat_classification"] = "gesundheit"
cats.loc[cats.id == 241, "stat_classification"] = "gesundheit"
cats.loc[cats.id == 215, "stat_classification"] = "gesundheit"
cats.loc[cats.id == 208, "stat_classification"] = "gesundheit"

cats.loc[cats.id == 7, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 218, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 238, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 235, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 233, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 188, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 207, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"
cats.loc[cats.id == 219, "stat_classification"] = "inneres-polizei-statistik-verwaltung-wohnen"

cats.loc[cats.id == 1, "stat_classification"] = "justiz"
cats.loc[cats.id == 227, "stat_classification"] = "justiz"

cats.loc[cats.id == 9, "stat_classification"] = "umwelt-naturschutz-atom"
cats.loc[cats.id == 216, "stat_classification"] = "umwelt-naturschutz-atom"
cats.loc[cats.id == 226, "stat_classification"] = "umwelt-naturschutz-atom"
cats.loc[cats.id == 217, "stat_classification"] = "umwelt-naturschutz-atom"
cats.loc[cats.id == 221, "stat_classification"] = "umwelt-naturschutz-atom"

cats.loc[cats.id == 18, "stat_classification"] = "verkehr-bau-stadtentwicklung"
cats.loc[cats.id == 190, "stat_classification"] = "verkehr-bau-stadtentwicklung"
cats.loc[cats.id == 10, "stat_classification"] = "verkehr-bau-stadtentwicklung"
cats.loc[cats.id == 225, "stat_classification"] = "verkehr-bau-stadtentwicklung"
cats.loc[cats.id == 206, "stat_classification"] = "verkehr-bau-stadtentwicklung"

cats.loc[cats.id == 198, "stat_classification"] = "digitales"
cats.loc[cats.id == 232, "stat_classification"] = "digitales"

cats.loc[cats.id == 5, "stat_classification"] = "militar"

cats.loc[cats.id == 187, "stat_classification"] = "wirtschaft-energie"
cats.loc[cats.id == 240, "stat_classification"] = "wirtschaft-energie"
cats.loc[cats.id == 224, "stat_classification"] = "wirtschaft-energie"
cats.loc[cats.id == 3, "stat_classification"] = "wirtschaft-energie"
cats.loc[cats.id == 204, "stat_classification"] = "wirtschaft-energie"
cats.loc[cats.id == 205, "stat_classification"] = "wirtschaft-energie"

cats.loc[cats.id == 14, "stat_classification"] = "finanzen"
cats.loc[cats.id == 228, "stat_classification"] = "finanzen"

cats.loc[cats.id == 210, "stat_classification"] = "europa"

cats.loc[cats.id == 229, "stat_classification"] = "oberste-landesbehorden"

In [7]:
cats.head()

,id,slug,stat_classification
0,181,arbeit,arbeit-soziales
1,12,auswartiges,auswaertiges
2,210,europa,europa
3,18,bau,verkehr-bau-stadtentwicklung
4,185,bildung,bildung-forschung


In [9]:
cats.to_csv(path_or_buf="categories.csv",index=False)

In [10]:
c = cats.loc[0]

In [64]:
cats.loc[0]

# ["id"]

id                                 181
slug                            arbeit
stat_classification    arbeit-soziales
Name: 0, dtype: object